In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null

E: Package 'python-software-properties' has no installation candidate


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [7]:
cd /content/drive/'My Drive'/'CIS 520 Project'

/content/drive/My Drive/CIS 520 Project


In [0]:
#imports
import warnings
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from sklearn.decomposition import PCA
from matplotlib import pyplot
from collections import Counter
import seaborn as sns

In [0]:
def get_data(csvfile):
  df = pd.read_pickle(csvfile)
  x, y = df['text'].tolist(), df['stars_x'].tolist()
  return x, y

In [10]:
#imports 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input
from keras.layers import LSTM, Conv1D, GlobalMaxPooling1D,MaxPooling1D, TimeDistributed
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from sklearn.naive_bayes import MultinomialNB 
import tensorflow as tf
tf.test.gpu_device_name()

Using TensorFlow backend.


'/device:GPU:0'

In [0]:
#TCN intialization cell
# https://github.com/titu1994/keras-neural-alu/blob/master/nac.py
from keras.engine import Layer
from keras.engine import InputSpec
from keras import initializers
from keras import regularizers
from keras import constraints
from keras import backend as K

from keras.utils.generic_utils import get_custom_objects


class NAC(Layer):
    def __init__(self, units,
                 kernel_W_initializer='glorot_uniform',
                 kernel_M_initializer='glorot_uniform',
                 kernel_W_regularizer=None,
                 kernel_M_regularizer=None,
                 kernel_W_constraint=None,
                 kernel_M_constraint=None,
                 **kwargs):
        """
        Neural Accumulator.
        # Arguments:
            units: Output dimension.
            kernel_W_initializer: Initializer for `W` weights.
            kernel_M_initializer: Initializer for `M` weights.
            kernel_W_regularizer: Regularizer for `W` weights.
            kernel_M_regularizer: Regularizer for `M` weights.
            kernel_W_constraint: Constraints on `W` weights.
            kernel_M_constraint: Constraints on `M` weights.
            epsilon: Small factor to prevent log 0.
        # Reference:
        - [Neural Arithmetic Logic Units](https://arxiv.org/abs/1808.00508)
        """
        super(NAC, self).__init__()
        self.units = units

        self.kernel_W_initializer = initializers.get(kernel_W_initializer)
        self.kernel_M_initializer = initializers.get(kernel_M_initializer)
        self.kernel_W_regularizer = regularizers.get(kernel_W_regularizer)
        self.kernel_M_regularizer = regularizers.get(kernel_M_regularizer)
        self.kernel_W_constraint = constraints.get(kernel_W_constraint)
        self.kernel_M_constraint = constraints.get(kernel_M_constraint)

        self.supports_masking = True

    def build(self, input_shape):
        assert len(input_shape) >= 2
        input_dim = input_shape[-1]

        self.W_hat = self.add_weight(shape=(input_dim, self.units),
                                     name='W_hat',
                                     initializer=self.kernel_W_initializer,
                                     regularizer=self.kernel_W_regularizer,
                                     constraint=self.kernel_W_constraint)

        self.M_hat = self.add_weight(shape=(input_dim, self.units),
                                     name='M_hat',
                                     initializer=self.kernel_M_initializer,
                                     regularizer=self.kernel_M_regularizer,
                                     constraint=self.kernel_M_constraint)

        self.input_spec = InputSpec(min_ndim=2, axes={-1: input_dim})
        self.built = True

    def call(self, inputs, **kwargs):
        W = K.tanh(self.W_hat) * K.sigmoid(self.M_hat)
        a = K.dot(inputs, W)
        return a

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) >= 2
        assert input_shape[-1]
        output_shape = list(input_shape)
        output_shape[-1] = self.units
        return tuple(output_shape)

    def get_config(self):
        config = {
            'units': self.units,
            'kernel_W_initializer': initializers.serialize(self.kernel_W_initializer),
            'kernel_M_initializer': initializers.serialize(self.kernel_M_initializer),
            'kernel_W_regularizer': regularizers.serialize(self.kernel_W_regularizer),
            'kernel_M_regularizer': regularizers.serialize(self.kernel_M_regularizer),
            'kernel_W_constraint': constraints.serialize(self.kernel_W_constraint),
            'kernel_M_constraint': constraints.serialize(self.kernel_M_constraint),
        }

        base_config = super(NAC, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


get_custom_objects().update({'NAC': NAC})

In [0]:
#preparing X and y!!!
csvfile = 'reviews_bow.csv'
reviews, stars = get_data(csvfile)
warnings.filterwarnings("ignore")
tempy, tempx = stars[:1000000], reviews[:1000000]
print('Data loaded')

#preparing y
y = np.array(tempy, dtype=np.uint8)
ymulti = y-1
y = to_categorical(ymulti, num_classes=5)
print('y prepared', y.shape)

#preparing x
vocab_size = 50000 #will use most frequent 50k words
num_words = 200 #97%coverage for reviews
tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(tempx)
sequences = tokenizer.texts_to_sequences(tempx)
padded_str = pad_sequences(sequences, maxlen = num_words)
#type changed here to use less memory 
x = np.array(padded_str, dtype=np.uint16)
# x = np.expand_dims(x, axis = 2)
print('x prepared', x.shape)

In [0]:
#class weights
# https://stackoverflow.com/questions/43481490/keras-class-weights-class-weight-for-one-hot-encoding
from sklearn.utils.class_weight import compute_class_weight

y_integers = np.argmax(y, axis=1)
class_weights = compute_class_weight('balanced', np.unique(y_integers), y_integers)
d_class_weights = dict(enumerate(class_weights))
print(d_class_weights)

In [0]:
def TCN_model():
  # create model
  # global d_class_weights
  i = Input(batch_shape=(None, 1, 200))
  # i = Input(batch_shape=(None, 44, 1024))
  o = NAC(2048, kernel_size=11, nb_stacks=2, dilations=[1, 2, 4, 8, 16], use_gating = True)(i)
  o = LSTM(200, return_sequences=False, dropout=0.25)(o)
  o = Dense(5, activation='sigmoid')(o)    
  model = Model(inputs=[i], outputs=[o])
  model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse','accuracy'])
  print(model.summary())
  return model

In [0]:
#CNN
#train-test
tempx = np.expand_dims(x, axis = 1)
tempy = to_categorical(ymulti, num_classes=5)
X_train, X_test, y_train, y_test = train_test_split(tempx, tempy, test_size=0.2, random_state=42, stratify = y)
print('X train shape: ', type(X_train), ' y train shape: ', type(y_train))
print('X train type: ', X_train.shape, ' y train type: ', y_train.shape)

model = TCN_model()
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.1, verbose = 1, class_weight=d_class_weights)
scores = model.evaluate(X_test, y_test, verbose=0)
model.save('tcn.h5')
pred = model.predict_classes(X_test)
y_int = [np.argmax(r) for r in y_test]
print(scores)
print(metrics.accuracy_score(y_int, pred))
print(metrics.classification_report(y_int, pred))